## Import

In [1]:
import os, shutil, random, pickle
from os.path import join
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from icecream import ic
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import ConfusionMatrixDisplay

from utils import get_angles
from feature_extractor import surflet_pairs_feature, get_histogram

PLY_PATH = './dataset/ply/test/pointCloud/pointCloud%d.ply'
PCA_THRESHOLD = 0.015

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Utils

In [2]:
def knn_predict(model, xs, labels, honv=True):
    if not honv: 
        return model.predict(xs)
    counts = [len(x) for x in xs]
    data = np.concatenate(xs, axis=0)
    probs = model.predict_proba(data)

    pred = []
    head = 0
    for count in counts:
        p = probs[head:head+count, :].sum(axis=0)
        pred.append(labels[p.argmax()])
        head += count
    return pred

In [3]:
def knn_inference(model, labels, honv=True, path=PLY_PATH):
    test_label = {}
    if not honv:
        xs = []
        for id in tqdm(range(1, 926)):
            xs.append(surflet_pairs_feature(filepath=path%id))
        xs = np.array(xs)
        ys = knn_predict(model, xs, labels=labels, honv=False)
    else:
        xs = []
        for id in tqdm(range(1, 926)):
            hists = get_histogram(filepath=path % id, bins=25, max_ratio=0.8)
            xs.append(np.array(hists))
        ys = knn_predict(model, xs, labels=labels, honv=True)

    for id, y in enumerate(ys, 1):
        test_label[id] = y
    return test_label

## Classify

### Plane vs others

In [4]:
test_label = {}
knn_12 = pickle.load(open('./honv/is_plane_25_08_03_full/model.pkl', 'rb'))
test_label = knn_inference(knn_12, [1, 2], honv=True)
del knn_12

  0%|          | 0/925 [00:00<?, ?it/s]

In [5]:
print('Remove plane false-accepts')
for id in range(1, 926):
    if test_label[id] == 1:
        x = np.asarray(o3d.io.read_point_cloud(PLY_PATH % id).points)
        model = PCA()
        model.fit_transform(x)
        # print(min(model.explained_variance_ratio_))
        if min(model.explained_variance_ratio_) > PCA_THRESHOLD:
            test_label[id] = 2

Remove plane false-accepts


### Cylinder-Cone vs. Sphere-Torus

In [6]:
knn_23 = pickle.load(open('./honv/2_class_25_08_05_full/model.pkl', 'rb'))
for id, label in knn_inference(knn_23, [2, 3], honv=True).items():
    if test_label[id] == 2:
        test_label[id] = label
del knn_23

  0%|          | 0/925 [00:00<?, ?it/s]

### Sphere vs. Torus

In [7]:
knn_35 = pickle.load(open('./SP/35_15_full/model.pkl', 'rb'))
for id, label in knn_inference(knn_35, [3, 5], honv=False).items():
    if test_label[id] == 3:
        test_label[id] = label
del knn_35

  0%|          | 0/925 [00:00<?, ?it/s]

/home/ccbien/works/projects/shrec22-track2/feature_extractor.py:17: RuntimeWarning: divide by zero encountered in double_scalars
  return np.arctan(y / x) + 2 * np.pi


### Cylinder vs. Cone

In [8]:
knn_24 = pickle.load(open('./honv/cyl-cone_25_08_05_full/model.pkl', 'rb'))
for id, label in knn_inference(knn_24, [2, 4], honv=True).items():
    if test_label[id] == 2:
        test_label[id] = label
del knn_24

  0%|          | 0/925 [00:00<?, ?it/s]

In [9]:
def foo():
    tmp = pickle.load(open('./res_dict/test_label_runA.pkl', 'rb'))
    tmp = test_label
    print(list(tmp.values()).count(1))
    print(list(tmp.values()).count(2))
    print(list(tmp.values()).count(3))
    print(list(tmp.values()).count(4))
    print(list(tmp.values()).count(5))

foo()

196
198
185
173
173


In [10]:
pickle.dump(test_label, open('./res_dict/test_label_runB.pkl', 'wb'))

In [12]:
name = ['', 'plane', 'cylinder', 'sphere', 'cone', 'torus']
for i in range(1, 926):
    print(name[test_label[i]])

sphere
cylinder
torus
sphere
sphere
cone
cylinder
sphere
sphere
sphere
cone
plane
sphere
cylinder
cone
torus
sphere
cylinder
sphere
cone
cone
cylinder
sphere
plane
plane
cylinder
plane
plane
sphere
torus
sphere
cylinder
cone
plane
plane
torus
torus
cylinder
torus
cone
torus
cylinder
sphere
cylinder
cone
cylinder
sphere
cylinder
cone
torus
plane
sphere
plane
sphere
cone
plane
sphere
cylinder
cylinder
cylinder
torus
torus
torus
plane
plane
cone
sphere
plane
sphere
plane
cone
torus
plane
sphere
torus
sphere
sphere
cone
plane
cylinder
plane
torus
sphere
plane
cylinder
plane
sphere
torus
cylinder
plane
plane
torus
plane
sphere
plane
torus
torus
cone
cone
cylinder
torus
plane
sphere
torus
cylinder
cone
plane
cone
cylinder
cylinder
cone
sphere
sphere
cylinder
sphere
cone
cone
cylinder
cylinder
cylinder
cylinder
cylinder
sphere
plane
sphere
plane
sphere
plane
torus
cylinder
torus
sphere
cone
plane
plane
torus
cylinder
cylinder
torus
sphere
plane
torus
sphere
sphere
cylinder
cylinder
sphere
tor